In [1]:
%reset -f

In [2]:
import pandas as pd
import numpy as np

In [3]:
pokemon_stat = pd.read_csv('pokemon_use.csv',index_col=0)
adv = pd.read_csv('advantage.csv',index_col=0)
combats=pd.read_csv('combats.csv')

In [4]:
# pokemon_stat.head()
# combats.head()

In [5]:
# new_combat_data contains the name of pokemons
cols = ["First_pokemon","Second_pokemon","Winner"]
new_combat_data=combats[cols].replace(pokemon_stat.Name)
# new_combat_data.head()

In [6]:
# Change combats dataframe to: First win 0, Second win 1
combats.Winner[combats.Winner == combats.First_pokemon] = 0
combats.Winner[combats.Winner == combats.Second_pokemon] = 1
# combats

In [7]:
def preprocess(pokemon_stat, combats):
    stats=["HP","Attack","Defense","Sp. Atk","Sp. Def","Speed","Legendary","Type 1"]
    stats1=["HP1","Attack1","Defense1","Sp. Atk1","Sp. Def1","Speed1","Legendary1","Type 1"]
    stats2=["HP2","Attack2","Defense2","Sp. Atk2","Sp. Def2","Speed2","Legendary2","Type 2"]
    stats_combined = stats1 + stats2
    final_stats = ["HP","Attack","Defense","Sp. Atk","Sp. Def","Speed","Legendary"]
    
    temp_list=[]
    adv_test_list=[]
    
    stats_df=pokemon_stat[stats].T.to_dict("list")
    one=combats.First_pokemon.map(stats_df)
    two=combats.Second_pokemon.map(stats_df)
    
    for i in range(len(one)):
        first = np.array(one[i][0:7])
        second = np.array(two[i][0:7])
        type1 = one[i][7]
        type2 = two[i][7]

        adv_coefficient = adv.loc[type1,type2]
        # adv_coefficient = adv.loc[(one.iloc[i])[7],(two.iloc[i])[7]]
        adv_test_list.append(adv_coefficient)

        first[1] = first[1] * adv_coefficient
        first[3] = first[3] * adv_coefficient
        # temp_list.append((first.tolist()+second.tolist()))
        
        # Calculate the difference
        temp_list.append(((first-second).tolist()))

    final_test = pd.DataFrame(temp_list, columns=final_stats)
    # print(new_test)

    return final_test

In [8]:
data = preprocess(pokemon_stat, combats)
# data.to_csv('processed_data.csv')

In [10]:
data = pd.concat([data,combats.Winner], axis=1)

In [11]:
data.head()

,HP,Attack,Defense,Sp. Atk,Sp. Def,Speed,Legendary,Winner
0,-20,-6,10,-15,10,-19,0,1
1,0,51,-18,108,39,0,0,1
2,-20,-35,10,-45,10,0,0,1
3,-37,-100,-50,-25,-50,-28,0,1
4,50,110,-105,220,-160,50,0,0


In [12]:
# separate to x and y
x_label=data.drop("Winner",axis=1)
y_label=data["Winner"]

x_label=x_label.values
y_laebl=y_label.values

In [13]:
# split train/test set
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x_label, y_label, test_size=0.20, random_state=10)

In [14]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
x_train = min_max_scaler.fit_transform(x_train)
x_test = min_max_scaler.transform(x_test)

In [15]:
x_test

array([[0.49438202, 0.32038835, 0.55172414, ..., 0.56354916, 0.64179104,
        0.5       ],
       [0.41573034, 0.46601942, 0.55632184, ..., 0.5323741 , 0.55522388,
        0.5       ],
       [0.46067416, 0.35728155, 0.44367816, ..., 0.44364508, 0.55223881,
        0.5       ],
       ...,
       [0.43820225, 0.38834951, 0.35632184, ..., 0.35971223, 0.49253731,
        0.        ],
       [0.49438202, 0.27572816, 0.50574713, ..., 0.53956835, 0.59701493,
        0.5       ],
       [0.50561798, 0.32038835, 0.47126437, ..., 0.6235012 , 0.50746269,
        0.5       ]])

In [ ]:
# For test CV grid search
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
# n_estimators = range(100,550,50)
n_estimators = [300]
# n_estimators = range(20,100,10)
hyper = {'n_estimators':n_estimators}
gd = GridSearchCV(estimator=RandomForestClassifier(random_state=0),param_grid=hyper,verbose=True,refit=True)
#estimator,param_grid, verbose 是GridSearchCV（）的参数，param_grid=hyper负责把先前设置的参数传给RandomForestClassifier()
X = x_train
y = y_train
model = gd.fit(X,y)
print(gd.best_score_)#输出最高的准确率
print(gd.best_estimator_)#输出最好的学习器，包括一系列参数
#得到最优参数后，再重新训练一次

In [ ]:
# 得到最优参数之后保存模型在这里写
# TODO:记录最优参数的数值，保存模型到一个文件中
# TODO：添加几行代码，直接读取模型文件，以便下一步求accuracy
# TODO
# gd = GridSearchCV(estimator=RandomForestClassifier(random_state=0),param_grid=hyper,verbose=True,refit=True)
# refit = True 
# gd返回最优参数训练的模型存入model中 model = gd.fit(X,y)

In [ ]:
# check on test set
pred = model.predict(x_test)
print('Accuracy of ', accuracy_score(pred, y_test)*100)

In [ ]:
# try to predict
# try_data=pd.read_csv('try.csv')

In [ ]:
# 此处只是可视化
# new_try_data=try_data[["First_pokemon","Second_pokemon"]].replace(pokemon_stat.Name)
# new_try_data.head()

In [ ]:
# final_try_data = preprocess(pokemon_stat, try_data)
# pred=model.predict(final_try_data)
# try_data["Winner"]=[try_data["First_pokemon"][i] if pred[i]==0 else try_data["Second_pokemon"][i] for i in range(len(pred))]

In [ ]:
# cols = ["First_pokemon","Second_pokemon","Winner"]
# combats_name = try_data[cols].replace(pokemon_stat.Name)
# combats_name[200:208]